In [ ]:
# Same Sigma Value for All 3 Dimensions
# Try Meshgrid type process
sigma = 5
dims = np.ndim(image_data)

# Compute LoG Kernel
w = np.ceil(5*sigma)
x = np.arange(-w, w, 1)

xrange = np.zeros([dims,np.size(x)])
for i in range(int(dims)):
    xrange[i,:] = x
    
x=np.zeros([dims,np.size(x),np.size(x),np.size(x)])
for i in range(int(dims)):
    x[i,:,:,:] = np.mgrid[np.arange(-w, w, 1)]

#x = np.meshgrid(x,x)
#print(np.shape(x))
print(np.shape(xrange))
print(xrange[1,0])
print(np.shape(x[0,:,:,:]))

####

In [ ]:
w = np.ceil(5*sigma)
x = np.arange(-w, w, 1)
a = np.mgrid[-w:w, -w:w, -w:w]
print(np.shape(a))
print(a[1,:,:,0])

In [ ]:
    /* Get number of FacesN */
    FacesDims = mxGetDimensions(prhs[0]);
    FacesN=FacesDims[0]*FacesDims[1];
    
    /* Get number of VertexN */
    VertexDims = mxGetDimensions(prhs[3]);
    VertexN=VertexDims[0]*VertexDims[1];
    
    outputdims[0]=VertexN;
    plhs[0]=mxCreateCellArray(1, outputdims);
    Ne=plhs[0];
    
    /* Neighborhs Unsorted */
    NU = (int **)malloc( VertexN* sizeof(int*) );
    NU_length = (int *)malloc( VertexN* sizeof(int) );
    for (i=0; i<VertexN; i++) {
        /* Set number of vertex neighbors to zero */
        NU_length[i]=0;
        /* Make room for neighbors */
        NU[i]=(int *)malloc( 20 * sizeof(int ) );
    }
    
    
    /* Loop throuh all faces */
    for (i=0; i<FacesN; i++) {
        /* Add the neighbors of each vertice of a face
         * to his neighbors list. */
        
        vertexa=(int)FacesA[i]-1; vertexb=(int)FacesB[i]-1; vertexc=(int)FacesC[i]-1;
        
        if(NU_length[vertexa]>18) { /* If need make extra room to store values */
            NU[vertexa]=(int *) realloc( NU[vertexa], (NU_length[vertexa]+2) * sizeof(int));
        }
        NU[vertexa][NU_length[vertexa]]  = vertexb+1; NU[vertexa][NU_length[vertexa]+1]= vertexc+1;
        NU_length[vertexa]+=2;
        
        if(NU_length[vertexb]>18) { /* If need make extra  store values */
            NU[vertexb]=(int *) realloc( NU[vertexb], (NU_length[vertexb]+2) * sizeof(int));
        }
        NU[vertexb][NU_length[vertexb]]  = vertexc+1; NU[vertexb][NU_length[vertexb]+1]=  vertexa+1;
        NU_length[vertexb]+=2;
        
        if(NU_length[vertexc]>18) { /* If need make extra store values */
            NU[vertexc]=(int *) realloc( NU[vertexc], (NU_length[vertexc]+2) * sizeof(int));
        }
        NU[vertexc][NU_length[vertexc]]  =  vertexa+1; NU[vertexc][NU_length[vertexc]+1]=  vertexb+1;
        NU_length[vertexc]+=2;
    }
    
    /*  Loop through all neighbor arrays and sort them (Rotation same as faces) */
    
    for (i=0; i<VertexN; i++) {
        if(NU_length[i]>0) {
            
            /* Create Matlab array for sorted neighbours of vertex*/
            Pneig=(int *)malloc(NU_length[i]*sizeof(int));
            PneighPos=0;
            Pneighf=NU[i];
            
            /* Start with the first vertex or if exist with a unique vertex */
            PneighStart=0;
            for(index1=0; index1<NU_length[i]; index1+=2) {
                found=0;
                for(index2=1; index2<NU_length[i]; index2+=2) {
                    if(Pneighf[index1]==Pneighf[index2]) {
                        found=1; break;
                    }
                }
                if(found==0) {
                    PneighStart=index1; break;
                }
            }
            
            Pneig[PneighPos]=Pneighf[PneighStart];    PneighPos++;
            Pneig[PneighPos]=Pneighf[PneighStart+1]; PneighPos++;
            
            
            /* Add the neighbours with respect to original rotation */
            for(j=1+found; j<(NU_length[i]/2); j++) {
                found=0;
                for(index1=0; index1<NU_length[i]; index1+=2) {
                    if(Pneighf[index1]==Pneig[PneighPos-1]) {
                        found2=0;
                        for(index2=0; index2<PneighPos; index2++) {
                            if(Pneighf[index1+1]==Pneig[index2]) { found2=1; }
                        }
                        if(found2==0) {
                            found=1;
                            Pneig[PneighPos]=Pneighf[index1+1];  PneighPos++;
                        }
                    }
                }
                if(found==0) /* This only happens with weird edge vertices */
                {
                    for(index1=0; index1<NU_length[i]; index1+=2) {
                        found2=0;
                        for(index2=0; index2<PneighPos; index2++) {
                            if(Pneighf[index1]==Pneig[index2]) { found2=1; }
                        }
                        if(found2==0) {
                            Pneig[PneighPos]=Pneighf[index1];  PneighPos++;
                            if(Pneighf[index1]==Pneig[PneighPos-1]) {
                                found2=0;
                                for(index2=0; index2<PneighPos; index2++) {
                                    if(Pneighf[index1+1]==Pneig[index2]) { found2=1; }
                                }
                                if(found2==0) {
                                    found=1;
                                    Pneig[PneighPos]=Pneighf[index1+1];  PneighPos++;
                                }
                            }
                        }
                        
                    }
                }
            }
            
            /* Add forgotten neigbours */
            if(PneighPos<NU_length[i]) {
                for(index1=0; index1<NU_length[i]; index1++) {
                    found2=0;
                    for(index2=0; index2<PneighPos; index2++) {
                        if(Pneighf[index1]==Pneig[index2]) { found2=1; break;}
                    }
                    if(found2==0) {
                        Pneig[PneighPos]=Pneighf[index1];  PneighPos++;
                    }
                }
            }
            
            outputdims[0]=PneighPos; /*(NU_length[i]/2);  */
            
            Pneig_matlab=mxCreateNumericArray(1, outputdims, mxDOUBLE_CLASS, mxREAL);
            Pneigd=(double*)mxGetPr(Pneig_matlab);

            
            /* Copy int to double array */
            for(j=0; j<outputdims[0]; j++) { Pneigd[j]=(double)Pneig[j]; }
            free(Pneig);
            mxSetCell(Ne, i, mxDuplicateArray(Pneig_matlab));
        }
    }
    
    /* Free memory */
    for (i=0; i<VertexN; i++) { free(NU[i]); }
    free(NU);
    
}

In [ ]:
#include "math.h"
#include "stdio.h"
#include "stdlib.h"
#include "string.h"

typedef wchar_t char16_t;

   /* Read all inputs (faces and vertices) */
   FacesA=mxGetPr(prhs[0]);
   FacesB=mxGetPr(prhs[1]);
   FacesC=mxGetPr(prhs[2]);
   VerticesX=mxGetPr(prhs[3]);
   VerticesY=mxGetPr(prhs[4]);
   VerticesZ=mxGetPr(prhs[5]);
   Iterations=mxGetPr(prhs[6]);
   Lambda=mxGetPr(prhs[7]);
   Ne=prhs[8];
   
    
   /* Get number of FacesN */
   FacesDims = mxGetDimensions(prhs[0]);   
   FacesN=FacesDims[0]*FacesDims[1];
   
   /* Get number of VertexN */
   VertexDims = mxGetDimensions(prhs[3]);   
   VertexN=VertexDims[0]*VertexDims[1];
   
   /* Intern vertices storage */
   VerticesNX = (double *)malloc( VertexN * sizeof(double) );
   VerticesNY = (double *)malloc( VertexN * sizeof(double) );
   VerticesNZ = (double *)malloc( VertexN * sizeof(double) );
   VerticesN2X = (double *)malloc( VertexN * sizeof(double) );
   VerticesN2Y = (double *)malloc( VertexN * sizeof(double) );
   VerticesN2Z = (double *)malloc( VertexN * sizeof(double) );
   
   /* Copy input arrays to ouput vertice arrays */
   memcpy( VerticesNX,VerticesX,VertexN* sizeof(double));
   memcpy( VerticesNY,VerticesY,VertexN* sizeof(double));
   memcpy( VerticesNZ,VerticesZ,VertexN* sizeof(double));
   
   for (j=0; j<Iterations[0]; j++)
   {
       for (i=0; i<VertexN; i++)
       {
             /* Get the neigbours of the vertice */
             PneigMatlab=mxGetCell(Ne, i);
             if(PneigMatlab == NULL)
             {
                 PneigLenght=-1;
             }
             else
             {
                 PneigDims=(mwSize *)mxGetDimensions(PneigMatlab);
                 PneigLenght=(PneigDims[0]*PneigDims[1]);
                 Pneig=(double *)mxGetPr(PneigMatlab);
             }
             W=1e-14; Ux=0; Uy=0; Uz=0;
             
             
             for(k=0; k<PneigLenght; k++)
             {
                 k1=k-1; if(k1<0) { k1+=PneigLenght; }
                 k2=k+1; if(k2>(PneigLenght-1)) { k2-=PneigLenght; };
                    
                 index0=i; index2=(int)Pneig[k]-1;
                 
                 /* Vertice of Left angle  */
                 index1=(int)Pneig[k1]-1;
                 
                 /* Calculate edge lengths  */
                 e0x=VerticesNX[index0]-VerticesNX[index1];  
                 e0y=VerticesNY[index0]-VerticesNY[index1];  
                 e0z=VerticesNZ[index0]-VerticesNZ[index1];
                 e1x=VerticesNX[index1]-VerticesNX[index2];  
                 e1y=VerticesNY[index1]-VerticesNY[index2];  
                 e1z=VerticesNZ[index1]-VerticesNZ[index2];
    
                 /* Normalize the edge vectors */
                 e0l = sqrt(e0x*e0x+e0y*e0y+e0z*e0z)+1e-14;
                 e0x/=e0l; e0y/=e0l; e0z/=e0l;
                 e1l = sqrt(e1x*e1x+e1y*e1y+e1z*e1z)+1e-14; 
                 e1x/=e1l; e1y/=e1l; e1z/=e1l;
    
                 /* Calculate angles of face seen from vertices */
                 Angle_Left= acos((e1x*e0x+e1y*e0y+e1z*e0z));
                         
                 /* Vertice of Right angle */
                 index1=(int)Pneig[k2]-1;

                 /* Calculate edge lengths  */
                 e0x=VerticesNX[index0]-VerticesNX[index1];  
                 e0y=VerticesNY[index0]-VerticesNY[index1];  
                 e0z=VerticesNZ[index0]-VerticesNZ[index1];
                 e1x=VerticesNX[index1]-VerticesNX[index2];  
                 e1y=VerticesNY[index1]-VerticesNY[index2];  
                 e1z=VerticesNZ[index1]-VerticesNZ[index2];
    
                 /* Normalize the edge vectors  */
                 e0l = sqrt(e0x*e0x+e0y*e0y+e0z*e0z)+1e-14;
                 e0x/=e0l; e0y/=e0l; e0z/=e0l;
                 e1l = sqrt(e1x*e1x+e1y*e1y+e1z*e1z)+1e-14; 
                 e1x/=e1l; e1y/=e1l; e1z/=e1l;
    
                 /* Calculate angles of face seen from vertices  */
                 Angle_Right= acos((e1x*e0x+e1y*e0y+e1z*e0z));
       
                 
                 Wt = Angle_Left+Angle_Right;

                 Ux+=Wt*(VerticesNX[index0]-VerticesNX[index2]);
                 Uy+=Wt*(VerticesNY[index0]-VerticesNY[index2]);
                 Uz+=Wt*(VerticesNZ[index0]-VerticesNZ[index2]);
                         
                 W += Wt;
            }
            
            VerticesN2X[i]=VerticesNX[i]-Lambda[0]*Ux/W;
            VerticesN2Y[i]=VerticesNY[i]-Lambda[0]*Uy/W;
            VerticesN2Z[i]=VerticesNZ[i]-Lambda[0]*Uz/W;             
       }
       
       /* Swap the variables */
       t=VerticesNX; VerticesNX=VerticesN2X; VerticesN2X=t;
       t=VerticesNY; VerticesNY=VerticesN2Y; VerticesN2Y=t;
       t=VerticesNZ; VerticesNZ=VerticesN2Z; VerticesN2Z=t;

       /* Swap output variable */
       if(swap==0) { swap=1; } else { swap=0; }
   }

   /* Create Output arrays for the new vertex coordinates */
   VertexNA[0]=VertexN;
   plhs[0] = mxCreateNumericArray(1, VertexNA, mxDOUBLE_CLASS, mxREAL);
   plhs[1] = mxCreateNumericArray(1, VertexNA, mxDOUBLE_CLASS, mxREAL);
   plhs[2] = mxCreateNumericArray(1, VertexNA, mxDOUBLE_CLASS, mxREAL);
   VerticesOX = mxGetPr(plhs[0]);
   VerticesOY=  mxGetPr(plhs[1]);
   VerticesOZ = mxGetPr(plhs[2]);

   if(swap==0)
   {
        /* Copy input arrays to ouput vertice arrays */
       memcpy( VerticesOX,VerticesN2X,VertexN* sizeof(double));
       memcpy( VerticesOY,VerticesN2Y,VertexN* sizeof(double));
       memcpy( VerticesOZ,VerticesN2Z,VertexN* sizeof(double));
   }
   else
   {
        /* Copy input arrays to ouput vertice arrays */
       memcpy( VerticesOX,VerticesNX,VertexN* sizeof(double));
       memcpy( VerticesOY,VerticesNY,VertexN* sizeof(double));
       memcpy( VerticesOZ,VerticesNZ,VertexN* sizeof(double));
   }
           
   
   /* Free memory */
   free(VerticesNX);
   free(VerticesNY);
   free(VerticesNZ);
   free(VerticesN2X);
   free(VerticesN2Y);
   free(VerticesN2Z);
}
 